In [2]:
!pip install torch torchvision
!pip install -U finance-datareader


In [3]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [61]:
x_frames = 7
start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*2))
end = datetime.date.today()
df_du = pdr.DataReader('POILDUBUSDM', 'fred', start=datetime.datetime.now() - datetime.timedelta(days=(100)), end=end)#석유
df_gold = pdr.DataReader('GOLDAMGBD228NLBM', 'fred', start=start, end=end)#금값
df_kospi = fdr.DataReader('KS11', start, end)#코스피
df_dollar = fdr.DataReader('USD/KRW', start, end)#원달러환율

#gold 채우기
df_gold = df_gold.fillna(value=-1)
gold = []
for i in df_gold.values:
    gold.append(i[0])

check = min(gold)
for i in range(0,len(gold)):
    if gold[i] == -1:
        gold[i] = check
    else:
        check = gold[i]

#oil 채우기
du = df_du.values[-1]
ddu = [du for i in range(x_frames)]

In [78]:
class predict_Dataset(Dataset):
    
    def __init__(self, symbol, x_frames,gold,du,df_kospi,df_dollar):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*2))
        self.end = datetime.date.today()

        self.scaler = MinMaxScaler()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)#주식 6가지 특징, 고,시,저,종,거,변
        
        if len(self.data.index) == 0:
            print(self.symbol,' is zero')
            return None
        self.data['dollar'] = df_dollar[['Close']]
        self.data['kospi'] = df_kospi[['Close']]
        self.data['oil'] = pd.DataFrame(data=ddu, index=self.data.index, columns=['oil'])
        self.data['gold'] = pd.DataFrame(data=gold, index=df_gold.index, columns=['gold'])
        self.data = pd.DataFrame(self.scaler.fit_transform(self.data))
        self.data = self.data.tail(x_frames)
        
    def __len__(self):
        return len(self.data) - self.x_frames + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx]
        data = data.values
        X = data[:self.x_frames]
        
        return X

In [66]:
class GRU(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(GRU, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        self.gru = nn.GRU(self.input_dim, self.hidden_dim*2, self.num_layers)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers)
        self.regressor = self.make_regressor()

    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim//2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim//2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        y_pred = self.regressor(x[-1].view(self.batch_size, -1))
        return y_pred

In [67]:
#주식정보들
Symbols = []
with open('/content/gdrive/My Drive/Symbols.txt', 'r') as f:
    data = f.readline()
    Symbols = data.split(',')
print(len(Symbols))

795


In [68]:
batch_size = 1
input_dim = 10
hidden_dim = 64
num_layers = 2
input_frame = 7
output_frame = 7
dropout = 1.0
use_bn = True

model = GRU(input_dim,hidden_dim,output_frame,num_layers,batch_size,dropout,use_bn)
modelPath = '/content/gdrive/My Drive/GRUmodel/GRU_Best_model.pt'
model.load_state_dict(torch.load(modelPath))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

GRU(
  (gru): GRU(10, 128, num_layers=2)
  (gru2): GRU(128, 64, num_layers=2)
  (regressor): Sequential(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=1.0, inplace=False)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  )
)

In [79]:
predicts = {}
i = 0
for sb in Symbols:
    i += 1
    if i % 100 == 0:
        print(i)
    pset = predict_Dataset(sb,input_frame,gold,du,df_kospi,df_dollar)

    model.eval()
    loader = DataLoader(pset,batch_size,shuffle=False)
    for X in loader:
        X = X.transpose(0, 1).float().to(device)
        y_pred = model(X)
        y_pred = y_pred.view(-1)
        predicts[sb] = y_pred[-1]


100
200
099340  is zero


ValueError: ignored

In [ ]:
!pip install pymysql
import pymysql

In [ ]:
conn = pymysql.connect(host='jukerdb.cwhsnjoqybdo.ap-northeast-2.rds.amazonaws.com', user='admin', password='',
                       db='WEB', charset='utf8')
 
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
 

for sb in result:
    #table : stockpreidct
    #symbol, value, type, day
    # SQL문 실행
    sql = """INSERT INTO stockpredict VALUES(%s,%s,%s,NOW())"""
    val = (sb,result[sb],'L')
    curs.execute(sql,val)
    conn.commit()

sql = "select * from stockpredict"
curs.execute(sql)
rows = curs.fetchall()
print(rows)     # 전체 rows

# Connection 닫기
conn.close()